# Continuous Training with Kubeflow Pipeline and Vertex AI

**Learning Objectives:**
1. Learn how to use KF pre-built components
1. Learn how to use KF lightweight python components
1. Learn how to build a KF pipeline with these components
1. Learn how to compile, upload, and run a KF pipeline


In this lab, you will build, deploy, and run a KFP pipeline that orchestrates the **Vertex AI** services to train, tune, and deploy a **scikit-learn** model.

## Setup

In [1]:
from google.cloud import aiplatform

In [3]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

qwiklabs-asl-00-61116abbd0d9


In [4]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


## Understanding the pipeline design


The workflow implemented by the pipeline is defined using a Python based Domain Specific Language (DSL). The pipeline's DSL is in the `pipeline_vertex/pipeline.py` file that we will generate below.

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.


### Build the trainer image

The training step in the pipeline will require a custom training container. The custom training image is defined in `trainer_image/Dockerfile`.

In [5]:
!cat trainer_image_vertex/Dockerfile

FROM us-docker.pkg.dev/vertex-ai/training/sklearn-cpu.1-0
RUN pip install -U fire cloudml-hypertune scikit-learn==1.2.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


Let's now build and push this trainer container to the Artifact Registry:

In [24]:
ARTIFACT_REGISTRY_DIR = "asl-artifact-repo"
IMAGE_NAME = "trainer_image_covertype_vertex"
IMAGE_TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"us-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REGISTRY_DIR}/{IMAGE_NAME}:{IMAGE_TAG}"
TRAINING_CONTAINER_IMAGE_URI

'us-docker.pkg.dev/qwiklabs-asl-00-61116abbd0d9/asl-artifact-repo/trainer_image_covertype_vertex:latest'

In [25]:
!gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI trainer_image_vertex

Creating temporary tarball archive of 4 file(s) totalling 6.7 KiB before compression.
Uploading tarball of [trainer_image_vertex] to [gs://qwiklabs-asl-00-61116abbd0d9_cloudbuild/source/1713949352.191655-2fb0727c96db4e058c1f86a295432159.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-00-61116abbd0d9/locations/global/builds/a2415326-4d9c-41eb-ab5c-4304e47a6917].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/a2415326-4d9c-41eb-ab5c-4304e47a6917?project=865563922071 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "a2415326-4d9c-41eb-ab5c-4304e47a6917"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-00-61116abbd0d9_cloudbuild/source/1713949352.191655-2fb0727c96db4e058c1f86a295432159.tgz#1713949352528002
Copying gs://qwiklabs-asl-00-61116abbd0d9_cloudbuild/source/1713949352.191655-2fb0727c96db4e058c1f86a295432159.tgz#1713949352528002...
/ [1 files][  1.8 KiB/  1.8 KiB]          

926e17c0ab1b: Layer already exists
a1741b153e96: Layer already exists
a8bc7d9be9d9: Layer already exists
59c56aee1fb4: Layer already exists
559421adb1cd: Pushed
latest: digest: sha256:d1bbbb141db1b84afe205eac84721bc3380ef3a0f8d7bdce0f1798f20ccf2062 size: 6171
DONE
--------------------------------------------------------------------------------
ID                                    CREATE_TIME                DURATION  SOURCE                                                                                                      IMAGES                                                                                                     STATUS
a2415326-4d9c-41eb-ab5c-4304e47a6917  2024-04-24T09:02:32+00:00  2M22S     gs://qwiklabs-asl-00-61116abbd0d9_cloudbuild/source/1713949352.191655-2fb0727c96db4e058c1f86a295432159.tgz  us-docker.pkg.dev/qwiklabs-asl-00-61116abbd0d9/asl-artifact-repo/trainer_image_covertype_vertex (+1 more)  SUCCESS


To match the ml framework version we use at training time while serving the model, we will have to supply the following serving container to the pipeline:

In [26]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"
)

**Note:** If you change the version of the training ml framework you'll have to supply a serving container with matchin version (see [pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)).

## Building and deploying the pipeline

Let us write the pipeline to disk:

### Exercise

Implement

1. the `train_and_deploy` function in the `pipeline_vertex/training_lightweight_component.py`
1. the `tune_hyperparameters` function in the `pipeline_vertex/tuning_lightweight_component.py`

and complete the TODOs in the `pipeline.py` file below:

In [27]:
%%writefile ./pipeline_vertex/pipeline.py
# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.
"""Kubeflow Covertype Pipeline."""
import os

from kfp import dsl
from training_lightweight_component import train_and_deploy
from tuning_lightweight_component import tune_hyperparameters

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

TRAINING_CONTAINER_IMAGE_URI = os.getenv("TRAINING_CONTAINER_IMAGE_URI")
SERVING_CONTAINER_IMAGE_URI = os.getenv("SERVING_CONTAINER_IMAGE_URI")

TRAINING_FILE_PATH = os.getenv("TRAINING_FILE_PATH")
VALIDATION_FILE_PATH = os.getenv("VALIDATION_FILE_PATH")

MAX_TRIAL_COUNT = int(os.getenv("MAX_TRIAL_COUNT", "5"))
PARALLEL_TRIAL_COUNT = int(os.getenv("PARALLEL_TRIAL_COUNT", "5"))
THRESHOLD = float(os.getenv("THRESHOLD", "0.6"))


@dsl.pipeline(
    name="covertype-kfp-pipeline",
    description="The pipeline training and deploying the Covertype classifier",
    pipeline_root=PIPELINE_ROOT,
)
def covertype_train(
    training_container_uri: str = TRAINING_CONTAINER_IMAGE_URI,
    serving_container_uri: str = SERVING_CONTAINER_IMAGE_URI,
    training_file_path: str = TRAINING_FILE_PATH,
    validation_file_path: str = VALIDATION_FILE_PATH,
    accuracy_deployment_threshold: float = THRESHOLD,
    max_trial_count: int = MAX_TRIAL_COUNT,
    parallel_trial_count: int = PARALLEL_TRIAL_COUNT,
    pipeline_root: str = PIPELINE_ROOT,
):
    staging_bucket = f"{pipeline_root}/staging"

    tuning_op =  tune_hyperparameters(
        project=PROJECT_ID,
        location=REGION,
        container_uri=training_container_uri,
        training_file_path=training_file_path,
        validation_file_path=validation_file_path,
        staging_bucket=staging_bucket,
        max_trial_count=max_trial_count,
        parallel_trial_count=parallel_trial_count,
    )

    accuracy = tuning_op.outputs["best_accuracy"]

    with dsl.Condition(
        accuracy >= accuracy_deployment_threshold, name="deploy_decision"
    ):
        train_and_deploy_op = train_and_deploy(
            project=PROJECT_ID,
            location=REGION,
            container_uri=training_container_uri,
            serving_container_uri=serving_container_uri,
            training_file_path=training_file_path,
            validation_file_path=validation_file_path,
            staging_bucket=staging_bucket,
            alpha=tuning_op.outputs["best_alpha"],
            max_iter=tuning_op.outputs["best_max_iter"],
        )
        

Overwriting ./pipeline_vertex/pipeline.py


### Compile the pipeline

Let stat by defining the environment variables that will be passed to the pipeline compiler:

In [28]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}

env: PIPELINE_ROOT=gs://qwiklabs-asl-00-61116abbd0d9-kfp-artifact-store/pipeline
env: PROJECT_ID=qwiklabs-asl-00-61116abbd0d9
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest
env: TRAINING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/qwiklabs-asl-00-61116abbd0d9/asl-artifact-repo/trainer_image_covertype_vertex:latest
env: TRAINING_FILE_PATH=gs://qwiklabs-asl-00-61116abbd0d9-kfp-artifact-store/data/training/dataset.csv
env: VALIDATION_FILE_PATH=gs://qwiklabs-asl-00-61116abbd0d9-kfp-artifact-store/data/validation/dataset.csv


Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [29]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-00-61116abbd0d9-kfp-artifact-store/


**Note:** In case the artifact store was not created and properly set before hand, you may need
to run in **CloudShell** the following command to allow Vertex AI to access it:

```
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)")
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
    --role="roles/storage.objectAdmin"
```

#### Use the CLI compiler to compile the pipeline

We compile the pipeline from the Python file we generated into a YAML description using the following command:

In [30]:
PIPELINE_YAML = "covertype_kfp_pipeline.yaml"

### Exercise

Compile the `pipeline_vertex/pipeline.py` with the `kfp dsl compile` command line:

In [34]:
!kfp dsl compile --py ./pipeline_vertex/pipeline.py --output $PIPELINE_YAML

/home/jupyter/asl-ml-immersion/notebooks/kubeflow_pipelines/pipelines/labs/covertype_kfp_pipeline.yaml


**Note:** You can also use the Python SDK to compile the pipeline from its python function

```python
compiler.Compiler().compile(
    pipeline_func=covertype_train, 
    package_path=PIPELINE_YAML,
)

```

The result is the pipeline file. 

In [32]:
!head {PIPELINE_YAML}

# PIPELINE DEFINITION
# Name: covertype-kfp-pipeline
# Description: The pipeline training and deploying the Covertype classifier
# Inputs:
#    accuracy_deployment_threshold: float [Default: 0.6]
#    max_trial_count: int [Default: 5.0]
#    parallel_trial_count: int [Default: 5.0]
#    pipeline_root: str [Default: 'gs://qwiklabs-asl-00-61116abbd0d9-kfp-artifact-store/pipeline']
#    serving_container_uri: str [Default: 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest']
#    training_container_uri: str [Default: 'us-docker.pkg.dev/qwiklabs-asl-00-61116abbd0d9/asl-artifact-repo/trainer_image_covertype_vertex:latest']


### Deploy the pipeline package

### Exercise

Upload and run the pipeline to Vertex AI using `aiplatform.PipelineJob`:

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="covertype_kfp_pipeline",
    template_path=PIPELINE_YAML,
    enable_caching=False,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/865563922071/locations/us-central1/pipelineJobs/covertype-kfp-pipeline-20240424091545
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/865563922071/locations/us-central1/pipelineJobs/covertype-kfp-pipeline-20240424091545')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/covertype-kfp-pipeline-20240424091545?project=865563922071
PipelineJob projects/865563922071/locations/us-central1/pipelineJobs/covertype-kfp-pipeline-20240424091545 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/865563922071/locations/us-central1/pipelineJobs/covertype-kfp-pipeline-20240424091545 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/865563922071/locations/us-central1/pipelineJobs/covertype-kfp-pipeline-20240424091545 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/86556392

### (Optional) Compile the custom components
If you want to easily reuse your custom components in other pipelines, consider compiling them into YAML format. <br>
Be aware that component YAML and pipeline YAML represent distinct objects.

In [ ]:
from kfp import compiler, components
from pipeline_vertex.training_lightweight_component import train_and_deploy
from pipeline_vertex.tuning_lightweight_component import tune_hyperparameters

compiler.Compiler().compile(
    train_and_deploy, "covertype_kfp_train_and_deploy.yaml"
)
compiler.Compiler().compile(
    tune_hyperparameters, "covertype_kfp_tune_hyperparameters.yaml"
)

To reuse a compiled component in another pipeline, easily load it using `kfp.components.load_component_from_file(<YAML PATH>)`.

In [ ]:
custom_component = components.load_component_from_file(
    "covertype_kfp_train_and_deploy.yaml"
)

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.